# SETUP

In [1]:
import os
os.getcwd()
os.chdir("..")

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [3]:
pd.options.display.max_rows = 10
pd.options.display.max_columns = 999999

# Data load

In [4]:
df = pd.read_pickle('data/processed/marketing_campaign_post_feature_engineering_and_cleaning.pickle')
df.head()

,year_birth,education,marital_status,income,kidhome,teenhome,dt_customer,recency,mntwines,mntfruits,mntmeatproducts,mntfishproducts,mntsweetproducts,mntgoldprods,numdealspurchases,numwebpurchases,numcatalogpurchases,numstorepurchases,numwebvisitsmonth,acceptedcmp3,acceptedcmp4,acceptedcmp5,acceptedcmp1,acceptedcmp2,complain,response,age,nb_kids,is_parent,adults,marital_status_cleaned,nb_family_members,education_cleaned,income_per_person,tenure_days,total_spendings,wines_share,fruits_share,meat_share,fish_share,sweets_share,gold_products_share,total_spendings_per_person,nb_accepted_campaigns,total_purchases,total_spendings_per_purchase,spendings_to_income,pct_deals_purchases,pct_web_purchasess,pct_catalog_purchases,pct_store_purchases
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5524,1957,Graduation,Alone,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,1,65,0,0,1.0,single,1,Postgraduate,58138,3612,1617,0.392703,0.054422,0.337662,0.106370,0.054422,0.054422,1617,0,22,73.500000,0.027813,0.136364,0.363636,0.454545,0.181818
2174,1954,Graduation,Alone,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,0,68,2,1,1.0,single,3,Postgraduate,15448,3062,27,0.407407,0.037037,0.222222,0.074074,0.037037,0.222222,9,0,4,6.750000,0.000583,0.500000,0.250000,0.250000,0.500000
4141,1965,Graduation,In couple,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,0,57,0,0,2.0,in couple,2,Postgraduate,35806,3261,776,0.548969,0.063144,0.163660,0.143041,0.027062,0.054124,388,0,20,38.800000,0.010836,0.050000,0.400000,0.100000,0.500000
6182,1984,Graduation,In couple,26646.0,1,0,2014-02-10,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,0,38,1,1,2.0,in couple,3,Postgraduate,8882,3088,53,0.207547,0.075472,0.377358,0.188679,0.056604,0.094340,17,0,6,8.833333,0.001989,0.333333,0.333333,0.000000,0.666667
5324,1981,PhD,In couple,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,0,41,1,1,2.0,in couple,3,Postgraduate,19431,3110,422,0.409953,0.101896,0.279621,0.109005,0.063981,0.035545,140,0,14,30.142857,0.007239,0.357143,0.357143,0.214286,0.428571


## Features selection and data preprocessing

In [5]:
num_features_to_scale = ["age", "tenure_days",
                         "recency", "income_per_person", "total_spendings", "total_spendings_per_person", "total_spendings_per_purchase",
                         "numwebvisitsmonth", "total_purchases",
                         "spendings_to_income"]

other_num_features = ["is_parent", "nb_family_members", 
                      "wines_share", "fruits_share", "meat_share", "fish_share", "sweets_share", "gold_products_share",
                      "pct_deals_purchases", "pct_web_purchasess", "pct_catalog_purchases", "pct_store_purchases",
                      "complain", "response", "nb_accepted_campaigns"]

cat_features = ["marital_status_cleaned", "education_cleaned"]

### Numeric features requiring scaling

In [6]:
df_num_to_scale = df.loc[:, num_features_to_scale]
df_num_to_scale.describe()

,age,tenure_days,recency,income_per_person,total_spendings,total_spendings_per_person,total_spendings_per_purchase,numwebvisitsmonth,total_purchases,spendings_to_income
count,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000
mean,53.195964,3302.863677,49.128700,24870.200448,607.046188,326.667713,38.183781,5.301794,12.565022,0.009925
std,11.987345,201.899680,28.963987,19611.285378,602.516120,431.469158,45.958297,2.370930,7.190046,0.016790
min,26.000000,2949.000000,0.000000,815.000000,8.000000,1.000000,2.666667,0.000000,1.000000,0.000093
25%,45.000000,3130.250000,24.000000,12184.250000,69.000000,24.000000,13.000000,3.000000,6.000000,0.002344
50%,52.000000,3305.000000,49.000000,18594.500000,396.500000,144.000000,29.688095,6.000000,12.000000,0.007973
75%,63.000000,3477.750000,74.000000,31916.750000,1046.500000,480.750000,49.156250,7.000000,18.000000,0.015535
max,129.000000,3648.000000,99.000000,222222.000000,2525.000000,2525.000000,1679.000000,20.000000,32.000000,0.706988


Outliers imputation

In [7]:
suspicious_num_vas = ['age', 'total_spendings', 'total_spendings_per_purchase']
df_num_to_scale[suspicious_num_vas] = df_num_to_scale[suspicious_num_vas].apply(lambda x: x.clip(*x.quantile([0.00, 0.99])))
df_num_to_scale.describe()

,age,tenure_days,recency,income_per_person,total_spendings,total_spendings_per_person,total_spendings_per_purchase,numwebvisitsmonth,total_purchases,spendings_to_income
count,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000
mean,53.118386,3302.863677,49.128700,24870.200448,605.273991,326.667713,37.301930,5.301794,12.565022,0.009925
std,11.700018,201.899680,28.963987,19611.285378,597.637945,431.469158,29.435469,2.370930,7.190046,0.016790
min,26.000000,2949.000000,0.000000,815.000000,8.000000,1.000000,2.666667,0.000000,1.000000,0.000093
25%,45.000000,3130.250000,24.000000,12184.250000,69.000000,24.000000,13.000000,3.000000,6.000000,0.002344
50%,52.000000,3305.000000,49.000000,18594.500000,396.500000,144.000000,29.688095,6.000000,12.000000,0.007973
75%,63.000000,3477.750000,74.000000,31916.750000,1046.500000,480.750000,49.156250,7.000000,18.000000,0.015535
max,77.000000,3648.000000,99.000000,222222.000000,2126.000000,2525.000000,128.626000,20.000000,32.000000,0.706988


Data scaling

In [8]:
df_scaled_num_features = df_num_to_scale.copy()

for i in df_scaled_num_features.columns:
    df_scaled_num_features.loc[:, i] = StandardScaler().fit_transform(np.array(df_scaled_num_features[[i]]))

df_scaled_num_features.describe()

,age,tenure_days,recency,income_per_person,total_spendings,total_spendings_per_person,total_spendings_per_purchase,numwebvisitsmonth,total_purchases,spendings_to_income
count,2.230000e+03,2.230000e+03,2.230000e+03,2.230000e+03,2.230000e+03,2.230000e+03,2.230000e+03,2.230000e+03,2.230000e+03,2.230000e+03
mean,2.712330e-16,3.405348e-17,7.136792e-17,1.389023e-17,5.790087e-17,-1.657867e-17,1.039029e-16,2.436019e-16,8.871827e-17,9.036120e-18
std,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00
min,-2.318327e+00,-1.753064e+00,-1.696580e+00,-1.226875e+00,-9.996152e-01,-7.549572e-01,-1.176915e+00,-2.236668e+00,-1.608838e+00,-5.857038e-01
25%,-6.940337e-01,-8.551395e-01,-8.677789e-01,-6.470150e-01,-8.975238e-01,-7.016390e-01,-8.257854e-01,-9.710580e-01,-9.132758e-01,-4.516047e-01
50%,-9.560981e-02,1.058348e-02,-4.444430e-03,-3.200763e-01,-3.494102e-01,-4.234571e-01,-2.587199e-01,2.945524e-01,-7.860161e-02,-1.162453e-01
75%,8.447706e-01,8.663983e-01,8.588900e-01,3.593915e-01,7.384487e-01,3.571909e-01,4.028126e-01,7.164225e-01,7.560726e-01,3.342547e-01
max,2.041618e+00,1.709828e+00,1.722224e+00,1.006543e+01,2.545131e+00,5.096136e+00,3.103214e+00,6.200734e+00,2.703646e+00,4.152644e+01


### Other numeric features - that do not require scaling

In [9]:
df_other_num = df.loc[:, other_num_features]
df_other_num.describe()

,is_parent,nb_family_members,wines_share,fruits_share,meat_share,fish_share,sweets_share,gold_products_share,pct_deals_purchases,pct_web_purchasess,pct_catalog_purchases,pct_store_purchases,complain,response,nb_accepted_campaigns
count,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000
mean,0.716592,2.599103,0.458770,0.049306,0.249714,0.071379,0.050739,0.120093,0.247970,0.329836,0.164667,0.505497,0.009417,0.148879,0.298206
std,0.450754,0.904998,0.228414,0.055716,0.126671,0.077915,0.061176,0.109194,0.356958,0.121800,0.140553,0.149548,0.096605,0.356049,0.678680
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.290197,0.008949,0.156318,0.012576,0.008637,0.038024,0.083333,0.250000,0.000000,0.401852,0.000000,0.000000,0.000000
50%,1.000000,3.000000,0.457822,0.029710,0.233746,0.048158,0.033333,0.085511,0.200000,0.333333,0.150000,0.500000,0.000000,0.000000,0.000000
75%,1.000000,3.000000,0.639277,0.069739,0.328238,0.103448,0.070291,0.169797,0.333333,0.400000,0.250000,0.600000,0.000000,0.000000,0.000000
max,1.000000,5.000000,0.963303,0.445545,0.997110,0.590909,0.945848,0.894150,15.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000


### Categorical features for encoding

In [10]:
df_cat = df.loc[:, cat_features]
df_cat.describe()

,marital_status_cleaned,education_cleaned
count,2230,2230
unique,2,2
top,in couple,Postgraduate
freq,1442,1973


In [11]:
lbl_encode = LabelEncoder()
df_cat_encoded = pd.DataFrame()
for i in cat_features:
    df_cat_encoded[i] = df_cat[[i]].apply(lbl_encode.fit_transform)
df_cat_encoded.describe()

,marital_status_cleaned,education_cleaned
count,2230.000000,2230.000000
mean,0.353363,0.115247
std,0.478121,0.319391
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,1.000000,0.000000
max,1.000000,1.000000


### Joining all preprocessed datasets

In [12]:
df_scaled = df_scaled_num_features.join(df_other_num).join(df_cat_encoded)
print(df.shape)
print(df_scaled.shape)

del df_scaled_num_features, df_other_num, df_cat_encoded
del df_num_to_scale, df_cat

(2230, 51)
(2230, 27)


In [13]:
df_scaled.head()

,age,tenure_days,recency,income_per_person,total_spendings,total_spendings_per_person,total_spendings_per_purchase,numwebvisitsmonth,total_purchases,spendings_to_income,is_parent,nb_family_members,wines_share,fruits_share,meat_share,fish_share,sweets_share,gold_products_share,pct_deals_purchases,pct_web_purchasess,pct_catalog_purchases,pct_store_purchases,complain,response,nb_accepted_campaigns,marital_status_cleaned,education_cleaned
id,,,,,,,,,,,,,,,,,,,,,,,,,,,
5524,1.015749,1.531482,0.306356,1.696740,1.693254,2.991226,1.230019,0.716422,1.312522,1.065679,0,1,0.392703,0.054422,0.337662,0.106370,0.054422,0.054422,0.136364,0.363636,0.454545,0.181818,0,1,0,1,0
2174,1.272216,-1.193254,-0.384312,-0.480556,-0.967816,-0.736412,-1.038162,-0.127318,-1.191501,-0.556537,1,3,0.407407,0.037037,0.222222,0.074074,0.037037,0.222222,0.500000,0.250000,0.250000,0.500000,0,0,0,1,0
4141,0.331836,-0.207395,-0.798712,0.557753,0.285732,0.142179,0.050905,-0.549188,1.034297,0.054294,0,2,0.548969,0.063144,0.163660,0.143041,0.027062,0.054124,0.050000,0.400000,0.100000,0.500000,0,0,0,0,0
6182,-1.292458,-1.064449,-0.798712,-0.815438,-0.924302,-0.717866,-0.967370,0.294552,-0.913276,-0.472750,1,3,0.207547,0.075472,0.377358,0.188679,0.056604,0.094340,0.333333,0.333333,0.000000,0.666667,0,0,0,0,0
5324,-1.035990,-0.955459,1.549558,-0.277413,-0.306733,-0.432730,-0.243267,-0.127318,0.199623,-0.159975,1,3,0.409953,0.101896,0.279621,0.109005,0.063981,0.035545,0.357143,0.357143,0.214286,0.428571,0,0,0,0,0


In [14]:
df_scaled.describe()

,age,tenure_days,recency,income_per_person,total_spendings,total_spendings_per_person,total_spendings_per_purchase,numwebvisitsmonth,total_purchases,spendings_to_income,is_parent,nb_family_members,wines_share,fruits_share,meat_share,fish_share,sweets_share,gold_products_share,pct_deals_purchases,pct_web_purchasess,pct_catalog_purchases,pct_store_purchases,complain,response,nb_accepted_campaigns,marital_status_cleaned,education_cleaned
count,2.230000e+03,2.230000e+03,2.230000e+03,2.230000e+03,2.230000e+03,2.230000e+03,2.230000e+03,2.230000e+03,2.230000e+03,2.230000e+03,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000,2230.000000
mean,2.712330e-16,3.405348e-17,7.136792e-17,1.389023e-17,5.790087e-17,-1.657867e-17,1.039029e-16,2.436019e-16,8.871827e-17,9.036120e-18,0.716592,2.599103,0.458770,0.049306,0.249714,0.071379,0.050739,0.120093,0.247970,0.329836,0.164667,0.505497,0.009417,0.148879,0.298206,0.353363,0.115247
std,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,0.450754,0.904998,0.228414,0.055716,0.126671,0.077915,0.061176,0.109194,0.356958,0.121800,0.140553,0.149548,0.096605,0.356049,0.678680,0.478121,0.319391
min,-2.318327e+00,-1.753064e+00,-1.696580e+00,-1.226875e+00,-9.996152e-01,-7.549572e-01,-1.176915e+00,-2.236668e+00,-1.608838e+00,-5.857038e-01,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-6.940337e-01,-8.551395e-01,-8.677789e-01,-6.470150e-01,-8.975238e-01,-7.016390e-01,-8.257854e-01,-9.710580e-01,-9.132758e-01,-4.516047e-01,0.000000,2.000000,0.290197,0.008949,0.156318,0.012576,0.008637,0.038024,0.083333,0.250000,0.000000,0.401852,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-9.560981e-02,1.058348e-02,-4.444430e-03,-3.200763e-01,-3.494102e-01,-4.234571e-01,-2.587199e-01,2.945524e-01,-7.860161e-02,-1.162453e-01,1.000000,3.000000,0.457822,0.029710,0.233746,0.048158,0.033333,0.085511,0.200000,0.333333,0.150000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,8.447706e-01,8.663983e-01,8.588900e-01,3.593915e-01,7.384487e-01,3.571909e-01,4.028126e-01,7.164225e-01,7.560726e-01,3.342547e-01,1.000000,3.000000,0.639277,0.069739,0.328238,0.103448,0.070291,0.169797,0.333333,0.400000,0.250000,0.600000,0.000000,0.000000,0.000000,1.000000,0.000000
max,2.041618e+00,1.709828e+00,1.722224e+00,1.006543e+01,2.545131e+00,5.096136e+00,3.103214e+00,6.200734e+00,2.703646e+00,4.152644e+01,1.000000,5.000000,0.963303,0.445545,0.997110,0.590909,0.945848,0.894150,15.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000,1.000000


In [15]:
df_scaled.to_pickle('data/processed/marketing_campaign_scaled_date_for_modelling.pickle')